In [1]:
pip install foolbox


     |████████████████████████████████| 1.7MB 4.5MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 163kB 28.9MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [74]:
import torch
from torchvision import transforms as T
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import *
from PIL import Image
import numpy as np


In [3]:
model = models.resnet18(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [26]:

transform = T.Compose([            
 T.Resize(256),                    
 T.CenterCrop(224),                
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])


img = Image.open('airplane2.jpg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)
input_img.shape

torch.Size([1, 3, 224, 224])

In [104]:
images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
#print(accuracy(fmodel, images, labels))
labels.dtype
x = torch.tensor([5])
labels.shape
x= labels[:1]
print(x)

PyTorchTensor(tensor([243]))


In [82]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [111]:
e = 0.5
attack = L2CarliniWagnerAttack()
for a in range(5):
  advs, _, success = attack(fmodel, input_img, x, epsilons=e)
  e = e + 1
  i = advs[0]
  advx = i.numpy()
  rescaled = (advx - advx.min()) / (advx.max() - advx.min())
  rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)
  rescaled = np.moveaxis(rescaled ,0, 1)
  output = Image.fromarray(rescaled)
  s = 'Repeated' + str(e) + '.png'
  output.save(s, 'png')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [57]:
def loadimage(s):
  img = Image.open(s)
  transformed_img = transform(img)
  final_img = transformed_img.unsqueeze(0)
  return final_img

In [92]:
name = "DDN5.1.png"
x = loadimage(name)
x.shape

torch.Size([1, 3, 224, 224])

In [93]:
modell = models.resnet18(pretrained=True).eval()
output = model(x)

In [94]:
_, index = torch.max(output, 1)
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())

airliner 93.52415466308594


In [95]:
_, indices = torch.sort(output, descending=True)
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('airliner', 93.52415466308594),
 ('wing', 5.847478866577148),
 ('warplane, military plane', 0.35963329672813416),
 ('space shuttle', 0.2405858188867569),
 ('airship, dirigible', 0.0106109119951725)]